In [1]:
import pickle
from detectors import *
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [2]:
def time_fit(det, data, labels):
    time_begin = time.time()
        
    det = det.fit(np.array(data), targets=np.array(labels))
    
    return det, time.time() - time_begin

def time_test(det, data):
    time_begin = time.time()
        
    result = det.predict_proba(np.array(data))
    
    return det, time.time() - time_begin, result


# load data

In [3]:
modes = ['bert_768', 'bow_50', 'bow_768']
subsets = {}
num_permutations = 10

result_pickle = 'data/results/amazon_same_dist.pickle'

for mode in modes:
    with open('data/movies/embeddings/amazon_{mode}_same_dist.pickle'.format(mode=mode), 'rb') as handle:
        subsets[mode] = permutations_embs, permutation_keys = pickle.load(handle)['data']

# initiate detectors

In [4]:
detectors = {
    'csdd': CosineSimilarityDriftDetector(),
    'kts' : KernelTwoSampleDriftDetector(),
    'aks' : AlibiKSDetector(),
    'ammd': AlibiMMDDetector(),
    'lsdd': AlibiLSDDDetector(),
    'cdbd': CDBDDetector()
}

# tests

In [ ]:

if os.path.isfile(result_pickle):  # Do not overwrite
    print('Loading result pickle: ', result_pickle)
    with open(result_pickle, 'rb') as handle:
        results = pickle.load(handle)
else:
    results = {mode: {detector: {} for detector in detectors} for mode in modes}

for detector in detectors:
    for mode in modes:
        if not detector in results[mode]:
            results[mode][detector] = {}
        if 'predictions' in results[mode][detector]: # skip already computed
            continue
        
        results[mode][detector]['predictions'] = []
        
        if detector == 'cdbd':
            fit_set = subsets[mode][0][0]
            for p in subsets[mode][0][num_permutations+1:]:
                fit_set.extend(p)
            fit_label = [x[1] for x in subsets[mode][1][0]]
            for p in subsets[mode][0][num_permutations+1:]:
                fit_set.extend(p)
        else:
            fit_set = subsets[mode][0][0]
            fit_label = [x[1] for x in subsets[mode][1][0]]
            for p in subsets[mode][1][num_permutations+1:]:
                fit_label.extend(p)
        
        det, t = time_fit(detectors[detector],
                          fit_set,
                          fit_label
                          )
        results[mode][detector]['time_fit'] = t
        results[mode][detector]['time_detect'] = []
        
        for permutation in subsets[mode][0][1:num_permutations+1]:
            det_2, t, res = time_test(det, permutation)
            results[mode][detector]['predictions'].append(res)
            results[mode][detector]['time_detect'].append(t)

with open(result_pickle, 'wb') as handle:
    pickle.dump(results, handle)

<ipython-input-2-66049519c172>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  det = det.fit(np.array(data), targets=np.array(labels))
<ipython-input-2-66049519c172>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  det = det.fit(np.array(data), targets=np.array(labels))
<ipython-input-2-66049519c172>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating

In [ ]:
print(results)